# Panorama

Image stitching is one of the most successful applications in
Computer Vision. Nowadays, it is hard to find a cell phone or an
image processing API that does not contain this
functionality(Panorama). This notebook guides you to create panoramic images.

In [1]:
#importing libraries
import cv2
import numpy as np

In [2]:
#read the images
image1 = cv2.imread(r'C:\Users\videe\Downloads\1.jpeg')
image2 = cv2.imread(r'C:\Users\videe\Downloads\2.jpeg')
copy1=image1.copy()
copy2=image2.copy()
#resize the images with suitable scale
image1 = cv2.resize(image1,(0,0), fx=0.5,fy=0.5, interpolation = cv2.INTER_AREA)
image2 = cv2.resize(image2,(0,0), fx=0.5,fy=0.5, interpolation = cv2.INTER_AREA)

In [3]:
#convert images into grayscale
image1_gray=cv2.cvtColor(image1, cv2.COLOR_RGB2GRAY)

image2_gray=cv2.cvtColor(image2, cv2.COLOR_RGB2GRAY)

## create descriptor and detect key points

In [4]:
#create object
orb = cv2.ORB_create()
#compute keypoints
keypoints1, descriptor1 = orb.detectAndCompute(image1_gray,None)
keypoints2, descriptor2 = orb.detectAndCompute(image2_gray,None)


In [5]:
#draw Keypoints
#use cv2.drawKeypoints()
keypoints_without_size = np.copy(image1)
keypoints_with_size = np.copy(image1)

cv2.drawKeypoints(image1, keypoints1, keypoints_without_size, color = (0, 255, 0))

cv2.drawKeypoints(image1, keypoints1, keypoints_with_size, flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)



array([[[174, 112,  74],
        [177, 115,  77],
        [177, 115,  77],
        ...,
        [191, 129,  83],
        [192, 127,  83],
        [192, 126,  85]],

       [[177, 115,  77],
        [178, 116,  78],
        [178, 116,  78],
        ...,
        [193, 130,  86],
        [194, 128,  87],
        [194, 127,  88]],

       [[178, 116,  78],
        [178, 116,  78],
        [177, 115,  77],
        ...,
        [195, 129,  88],
        [194, 127,  88],
        [194, 126,  89]],

       ...,

       [[158, 174, 190],
        [158, 174, 190],
        [157, 173, 189],
        ...,
        [158, 169, 178],
        [160, 172, 181],
        [160, 172, 181]],

       [[154, 172, 189],
        [153, 171, 188],
        [152, 170, 187],
        ...,
        [134, 151, 156],
        [135, 151, 156],
        [135, 151, 156]],

       [[129, 149, 167],
        [136, 156, 174],
        [133, 153, 171],
        ...,
        [165, 181, 181],
        [162, 178, 178],
        [162, 178, 178]]

In [77]:
cv2.imshow('image1',copy1)
cv2.imshow('image2',copy2)
cv2.waitKey(0)
cv2.destroyAllWindows()

references:- https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_feature2d/py_matcher/py_matcher.html

In [78]:
#create a bruteforce matcher object
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
#match the descriptors using bf.match() 
matches=bf.match(descriptor1,descriptor2)
#sort matches
matches=sorted(matches, key = lambda x : x.distance)
#Select first few matches from sorted matches
#because having a large number of matches cause RANSAC algorithm to overfit the data
matches=matches[:int(len(matches)*0.3)]
#draw matches
result=cv2.drawMatches(image1,keypoints1,image2_gray,keypoints2,matches,image2_gray, flags = 2)

cv2.imshow("result",result)



The result of matches = bf.match(des1,des2) line is a list of DMatch objects. This DMatch object has following attributes:

DMatch.distance - Distance between descriptors. The lower, the better it is.
DMatch.trainIdx - Index of the descriptor in train descriptors
DMatch.queryIdx - Index of the descriptor in query descriptors
DMatch.imgIdx - Index of the train image.

So, Now we need to extract the points and feed them to cv2.findhomography function

In [79]:
#finding homography

points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

h,mask = cv2.findHomography(points1,points2,cv2.RANSAC,5.0)

In [81]:
#wrap the image
dst = cv2.warpPerspective(image1,h,(image1.shape[1] + image2.shape[0], image2.shape[0]))
cv2.imshow('Wrapped',dst)
dst[0:image2.shape[0], 0:image2.shape[1]] = image2
cv2.imshow('stitched',dst)

cv2.waitKey(0)
cv2.destroyAllWindows()